# 浙江杭州市萧山区项目`WRF-CMAQ`模拟分析
## 观测结果预处理

---
*@author: Evan*\
*@date: 2023-06-20*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [3]:
xls=pd.read_excel('F:/Data/Project_Xiaoshan/202209/obsdata/hangzhou.xlsx',header=0,na_values=['—',-99])
xls

,Year,Month,Day,Hour,AQI,PM2.5,PM2.5_24h,PM10,PM10_24h,SO2,SO2_24h,NO2,NO2_24h,O3,O3_24h,O3_8h,O3_8h_24h,CO,CO_24h,Station
0,2019,1,1,0,75.0,55.0,45.0,62.0,52.0,9.0,9.0,59.0,51.0,4.0,24.0,4.0,20.0,1.0,1.0,1
1,2019,1,1,1,72.0,52.0,46.0,58.0,52.0,8.0,9.0,56.0,52.0,4.0,24.0,4.0,4.0,1.0,1.0,1
2,2019,1,1,2,63.0,45.0,46.0,50.0,53.0,8.0,9.0,53.0,53.0,5.0,23.0,4.0,4.0,1.0,1.0,1
3,2019,1,1,3,58.0,41.0,46.0,46.0,53.0,10.0,9.0,45.0,53.0,11.0,23.0,7.0,7.0,1.0,1.0,1
4,2019,1,1,4,65.0,47.0,46.0,51.0,54.0,10.0,9.0,48.0,53.0,8.0,23.0,7.0,7.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521131,2023,3,31,19,36.0,21.0,31.0,36.0,47.0,6.0,6.0,45.0,33.0,63.0,101.0,75.0,80.0,0.7,0.7,14
521132,2023,3,31,20,38.0,22.0,31.0,38.0,47.0,6.0,6.0,39.0,34.0,73.0,99.0,76.0,80.0,0.6,0.7,14
521133,2023,3,31,21,51.0,33.0,32.0,51.0,48.0,6.0,6.0,52.0,35.0,54.0,99.0,72.0,80.0,0.7,0.7,14
521134,2023,3,31,22,55.0,39.0,32.0,60.0,49.0,7.0,6.0,66.0,37.0,34.0,99.0,66.0,80.0,0.8,0.7,14


In [11]:
pd.to_datetime('2019-1-2 00')

Timestamp('2019-01-02 00:00:00')

In [12]:
# 读取原始数据
# xls=pd.read_excel('F:/Data/Project_Xiaoshan/202209/obsdata/hangzhou.xlsx',header=0,na_values=['—',-99])

# 将日期与时间列合并
xls['datetime'] = pd.to_datetime(xls['Year'].astype(str)+'-'+xls['Month'].astype(str)+'-'+xls['Day'].astype(str)+' '+xls['Hour'].apply(lambda x: '{0:0>2}'.format(x)))

# 删除“日期”列和“时间”列
xls.drop(['Year','Month','Day','Hour'], axis=1, inplace=True)

# 设置为索引
xls.set_index('datetime',inplace=True)
xls

,AQI,PM2.5,PM2.5_24h,PM10,PM10_24h,SO2,SO2_24h,NO2,NO2_24h,O3,O3_24h,O3_8h,O3_8h_24h,CO,CO_24h,Station
datetime,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,75.0,55.0,45.0,62.0,52.0,9.0,9.0,59.0,51.0,4.0,24.0,4.0,20.0,1.0,1.0,1
2019-01-01 01:00:00,72.0,52.0,46.0,58.0,52.0,8.0,9.0,56.0,52.0,4.0,24.0,4.0,4.0,1.0,1.0,1
2019-01-01 02:00:00,63.0,45.0,46.0,50.0,53.0,8.0,9.0,53.0,53.0,5.0,23.0,4.0,4.0,1.0,1.0,1
2019-01-01 03:00:00,58.0,41.0,46.0,46.0,53.0,10.0,9.0,45.0,53.0,11.0,23.0,7.0,7.0,1.0,1.0,1
2019-01-01 04:00:00,65.0,47.0,46.0,51.0,54.0,10.0,9.0,48.0,53.0,8.0,23.0,7.0,7.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 19:00:00,36.0,21.0,31.0,36.0,47.0,6.0,6.0,45.0,33.0,63.0,101.0,75.0,80.0,0.7,0.7,14
2023-03-31 20:00:00,38.0,22.0,31.0,38.0,47.0,6.0,6.0,39.0,34.0,73.0,99.0,76.0,80.0,0.6,0.7,14
2023-03-31 21:00:00,51.0,33.0,32.0,51.0,48.0,6.0,6.0,52.0,35.0,54.0,99.0,72.0,80.0,0.7,0.7,14


In [13]:
# 将所有因包含括号等字符被读取为字符串的数据设置为空值
xls = xls.replace(to_replace=[r'\(.*?\)'], value=np.nan, regex=True)

In [14]:
sep2022 = xls.loc['2022-09']
sep2022

,AQI,PM2.5,PM2.5_24h,PM10,PM10_24h,SO2,SO2_24h,NO2,NO2_24h,O3,O3_24h,O3_8h,O3_8h_24h,CO,CO_24h,Station
datetime,,,,,,,,,,,,,,,,
2022-09-01 00:00:00,40.0,27.0,16.0,40.0,29.0,6.0,6.0,17.0,19.0,70.0,119.0,87.0,97.0,0.6,0.6,1
2022-09-01 01:00:00,26.0,8.0,16.0,23.0,29.0,5.0,6.0,11.0,18.0,82.0,119.0,82.0,82.0,0.4,0.6,1
2022-09-01 02:00:00,25.0,6.0,16.0,21.0,29.0,6.0,6.0,14.0,18.0,79.0,119.0,80.0,82.0,0.4,0.6,1
2022-09-01 03:00:00,22.0,9.0,16.0,22.0,28.0,6.0,6.0,16.0,18.0,70.0,119.0,77.0,82.0,0.4,0.6,1
2022-09-01 04:00:00,27.0,17.0,16.0,27.0,29.0,6.0,6.0,18.0,19.0,61.0,119.0,73.0,82.0,0.5,0.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30 19:00:00,42.0,24.0,30.0,42.0,51.0,8.0,8.0,28.0,43.0,126.0,212.0,174.0,174.0,0.6,0.7,14
2022-09-30 20:00:00,41.0,23.0,30.0,41.0,51.0,8.0,8.0,30.0,42.0,103.0,212.0,170.0,174.0,0.6,0.7,14
2022-09-30 21:00:00,40.0,22.0,30.0,40.0,51.0,8.0,8.0,37.0,41.0,68.0,212.0,159.0,174.0,0.6,0.7,14


In [15]:
sep2022.to_excel('D:/Download/hz_202209.xlsx',index=True)